In [70]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import gspread
from google.colab import auth

#First Authenticate google colab
auth.authenticate_user()

#Authenticate google sheet
from google.auth import default
creds, _ = default()

gs = gspread.authorize(creds)

#Get google sheet
sh = gs.open("Dados compilados sobre aves catalogadas no Brasil")
sh_regions = sh.worksheet("Regiões com espécies em comum com o Brasil")
sh_birds = sh.worksheet("Espécies de aves no Brasil")
sh_birds_states = sh.worksheet("Espécies de aves por estado do Brasil")
sh_brazil_states = sh.worksheet("Estados")

# **Regiões do mundo com espécies em comum com o Brasil**

In [ ]:
#Extract html
page_regions = requests.get('https://avibase.bsc-eoc.org/compare_regions.jsp?region=BR&list=clements')
bs = BeautifulSoup(page_regions.text, 'html.parser')

In [ ]:
#Transform table into list
tb_regions = bs.find(attrs={'id':'compareRegions'}).find_all('tbody')[0].find_all('tr')
tb_regions = [ [td.string for td in tr.find_all('td')] for tr in tb_regions]

tb_regions_headers = bs.find(attrs={'id':'compareRegions'}).find('thead').find('tr').find_all('th')
tb_regions_headers = [ x.string for x in tb_regions_headers ]

tb_regions = [tb_regions_headers] + tb_regions

In [ ]:
#Filter columns with pandas
df = pd.DataFrame(tb_regions[1:], columns=tb_regions[0])

df = df[['Region2','Shared','% shared/region1','%shared/region2']]

df[['Shared','% shared/region1','%shared/region2']].apply(pd.to_numeric)

data = df.fillna(-1).values.tolist()

In [ ]:
#Update Google sheet
sh_regions.append_rows(data,table_range='A2')

{'spreadsheetId': '1jkUNttrcJa_U75cVSiNhHXGSfWZbFmdK8E5CcJb7XMM',
 'updates': {'spreadsheetId': '1jkUNttrcJa_U75cVSiNhHXGSfWZbFmdK8E5CcJb7XMM',
  'updatedRange': "'Regiões com espécies em comum com o Brasil'!A2:D243",
  'updatedRows': 242,
  'updatedColumns': 4,
  'updatedCells': 968}}

# **Espécies de aves por estados do Brasil**

In [68]:
states = sh_brazil_states.col_values(1)[1:] # codes of states
birds = [['Nome científico', 'Nome popular', 'Família', 'Situação', 'Estado']]

for state in states:
  # Extract html of each state
  page = requests.get(f"https://avibase.bsc-eoc.org/checklist.jsp?region=BR{state}")
  bs = BeautifulSoup(page.text, 'html.parser')

  # transform table into lists and append to birds
  if bs.find('table'):
    tb = bs.find('table').find_all('tr')

    family = ''
    for row in tb:
      if row.has_attr('valign'):
        family = row.find("td").get_text().strip()
      else:
        row_content = [ col.get_text().strip() for col in row.find_all('td') ]
        row_content.insert(2,family)
        row_content.append(state)
        birds.append(row_content)

In [71]:
# Filter columns with pandas
df = pd.DataFrame(birds[1:], columns=birds[0])

df_birds = df[['Nome científico', 'Nome popular', 'Família']].drop_duplicates()
df_birds_states = df[['Nome científico','Estado','Situação']].drop_duplicates()

data_birds = df_birds.fillna(-1).values.tolist()
data_birds_states = df_birds_states.fillna(-1).values.tolist()

In [72]:
# Update Google sheet
sh_birds.append_rows(data_birds,table_range='A2')
sh_birds_states.append_rows(data_birds_states,table_range='A2')

{'spreadsheetId': '1jkUNttrcJa_U75cVSiNhHXGSfWZbFmdK8E5CcJb7XMM',
 'updates': {'spreadsheetId': '1jkUNttrcJa_U75cVSiNhHXGSfWZbFmdK8E5CcJb7XMM',
  'updatedRange': "'Espécies de aves por estado do Brasil'!A2:C16735",
  'updatedRows': 16734,
  'updatedColumns': 3,
  'updatedCells': 50202}}